In [1]:
import os
import requests
import datetime
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
options = Options()
browser = Chrome(ChromeDriverManager().install(), options=options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 110.0.5481
[WDM] - Get LATEST driver version for 110.0.5481
[WDM] - Driver [/Users/natalie/.wdm/drivers/chromedriver/mac64/110.0.5481.77/chromedriver] found in cache


In [3]:
browser.maximize_window()
time.sleep(5)

## Collect All Reviews for a Reviewer

In [4]:
userhandle = "juliuslim" ### TO CHANGE HERE
URL = "https://www.burpple.com/@" + userhandle + "/timeline"

In [5]:
browser.get(URL)

In [6]:
load_more = True
page_count = 0
while load_more:
    if (page_count % 100 == 0):
        print(page_count)
    try:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        more_reviews = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.ID, "masonryViewMore-btn")))
        more_reviews.send_keys(Keys.ENTER)
        time.sleep(3)
        page_count +=1
    except Exception as e:
        load_more = False
        print(e)

0


In [ ]:
# find all reviews by a given revier
page_html = BeautifulSoup(browser.page_source, "html.parser")
all_reviews = page_html.findAll(class_="food card feed-item")

In [ ]:
len(all_reviews)

In [ ]:
reviews_by_reviewer = []
for review_listing in all_reviews:
    # name and address
    header = review_listing.find(class_="food-place card-item card-item--header")
    try: 
        name = header.find(class_="card-item-set--link-title").text
        name = name.replace('\n', '')

        address = review_listing.find(class_="card-item-set--link-subtitle").text
        address = address.replace('\n', '')
    except:
        name = None
        address = None

    # review
    review = review_listing.find(class_="food-description").text

    # date
    user_card = review_listing.find(class_="food-user card-item")
    try:
        date = user_card.find(class_="card-item-set--link-subtitle").text
        date = date.replace('\n', '')
    except:
        date = None

    # restaurant listing link
    link = review_listing.find_all("a", href=True)[0]['href']
    link = 'https://www.burpple.com' + link

    reviews_by_reviewer.append([name, address, review, date, link])

In [ ]:
reviews_by_reviewer_df = pd.DataFrame(reviews_by_reviewer, columns=['name', 'address', 'review', 'date', 'link'])

In [ ]:
reviews_by_reviewer_df

In [ ]:
reviews_by_reviewer_df.to_csv("reviewer-data/" + userhandle + "_reviews.csv")